In [1]:
import numpy as np
import pandas as pd
import sklearn
import json
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics

from sklearn.model_selection import KFold 
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import FeatureUnion
from scipy.sparse import coo_matrix, hstack, vstack
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
from sklearn.metrics import f1_score

## Importando DataSet

In [2]:
filename = 'GBvideos.csv'
category_filename = 'GB_category_id.json'

videos = pd.read_csv(filename)
videos = videos.dropna(axis=0)
print("DataSet null inputs: \n"+str(videos.isnull().sum())+"\n")
print(filename+" shape: "+str(videos.shape))
videos.head()

DataSet null inputs: 
video_id                  0
trending_date             0
title                     0
channel_title             0
category_id               0
publish_time              0
tags                      0
views                     0
likes                     0
dislikes                  0
comment_count             0
thumbnail_link            0
comments_disabled         0
ratings_disabled          0
video_error_or_removed    0
description               0
dtype: int64

GBvideos.csv shape: (38304, 16)


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,Jw1Y-zhQURU,17.14.11,John Lewis Christmas Ad 2017 - #MozTheMonster,John Lewis,26,2017-11-10T07:38:29.000Z,"christmas|""john lewis christmas""|""john lewis""|...",7224515,55681,10247,9479,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,False,False,False,Click here to continue the story and make your...
1,3s1rvMFUweQ,17.14.11,Taylor Swift: …Ready for It? (Live) - SNL,Saturday Night Live,24,2017-11-12T06:24:44.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",1053632,25561,2294,2757,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,False,False,False,Musical guest Taylor Swift performs …Ready for...
2,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787420,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
3,PUTEiSjKwJU,17.14.11,Goals from Salford City vs Class of 92 and Fri...,Salford City Football Club,17,2017-11-13T02:30:38.000Z,"Salford City FC|""Salford City""|""Salford""|""Clas...",27833,193,12,37,https://i.ytimg.com/vi/PUTEiSjKwJU/default.jpg,False,False,False,Salford drew 4-4 against the Class of 92 and F...
4,rHwDegptbI4,17.14.11,Dashcam captures truck's near miss with child ...,Cute Girl Videos,25,2017-11-13T01:45:13.000Z,[none],9815,30,2,30,https://i.ytimg.com/vi/rHwDegptbI4/default.jpg,False,False,False,Dashcam captures truck's near miss with child ...


# Tratamento do DataSet

## Correção de formato de data-hora para padrão Unix

In [3]:
videos['trending_date'] = pd.to_datetime(videos['trending_date'], format='%y.%d.%m')
videos['publish_time'] = pd.to_datetime(videos['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')

# separates date and time into two columns from 'publish_time' column

videos.insert(4, 'publish_date', videos['publish_time'].dt.date)
videos['publish_time'] = videos['publish_time'].dt.time
videos['publish_date']=pd.to_datetime(videos['publish_date'])

videos.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,Jw1Y-zhQURU,2017-11-14,John Lewis Christmas Ad 2017 - #MozTheMonster,John Lewis,2017-11-10,26,07:38:29,"christmas|""john lewis christmas""|""john lewis""|...",7224515,55681,10247,9479,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,False,False,False,Click here to continue the story and make your...
1,3s1rvMFUweQ,2017-11-14,Taylor Swift: …Ready for It? (Live) - SNL,Saturday Night Live,2017-11-12,24,06:24:44,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",1053632,25561,2294,2757,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,False,False,False,Musical guest Taylor Swift performs …Ready for...
2,n1WpP7iowLc,2017-11-14,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,2017-11-10,10,17:00:03,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787420,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
3,PUTEiSjKwJU,2017-11-14,Goals from Salford City vs Class of 92 and Fri...,Salford City Football Club,2017-11-13,17,02:30:38,"Salford City FC|""Salford City""|""Salford""|""Clas...",27833,193,12,37,https://i.ytimg.com/vi/PUTEiSjKwJU/default.jpg,False,False,False,Salford drew 4-4 against the Class of 92 and F...
4,rHwDegptbI4,2017-11-14,Dashcam captures truck's near miss with child ...,Cute Girl Videos,2017-11-13,25,01:45:13,[none],9815,30,2,30,https://i.ytimg.com/vi/rHwDegptbI4/default.jpg,False,False,False,Dashcam captures truck's near miss with child ...


## Tratando para pegar somente a ultima entrada para cada video (a entrada mais atualizada)

In [4]:
videos_lastentry = videos.sort_values('trending_date', ascending=False).drop_duplicates(['video_id'])
print(filename+" shape: "+str(videos_lastentry.shape))
videos_lastentry.head()

GBvideos.csv shape: (3217, 17)


,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
38915,4YFo4bdMO8Q,2018-06-14,KYLE - Ikuyo feat. 2 Chainz & Sophia Black [A...,SuperDuperKyle,2018-05-11,10,04:06:35,"Kyle|""SuperDuperKyle""|""Ikuyo""|""2 Chainz""|""Soph...",607552,18271,274,1423,https://i.ytimg.com/vi/4YFo4bdMO8Q/default.jpg,False,False,False,Debut album 'Light of Mine' out now: http://ky...
38811,3KmIMuLh4Vs,2018-06-14,Interpol - The Rover,InterpolVEVO,2018-06-07,10,16:29:06,"Interpol|""The Rover""|""Marauder""|""Matador""|""Mat...",442492,17004,178,1670,https://i.ytimg.com/vi/3KmIMuLh4Vs/default.jpg,False,False,False,'The Rover' is the first single taken from Int...
38809,CWSYOdxx8Tg,2018-06-14,How To Make a Hit Football Anthem,Brett Domino,2018-06-07,23,07:30:00,"brett domino|""Umbro""|""England Anthem""|""World C...",56138,2814,73,170,https://i.ytimg.com/vi/CWSYOdxx8Tg/default.jpg,False,False,False,Brett Domino and Umbro’s guide to writing a hi...
38808,RtvUoab88UE,2018-06-14,Dear Ms. Banks,Bob The Drag Queen,2018-06-06,22,22:27:21,"bob the drag queen|""drag""|""diss track""|""rap""|""...",169099,13780,795,2084,https://i.ytimg.com/vi/RtvUoab88UE/default.jpg,False,False,False,Thanks to Mitch Ferrino for helping with audio.
38807,11K013qpRR4,2018-06-14,The LEGO Movie 2 Trailer (2019),FilmSelect Trailer,2018-06-05,24,12:50:30,"trailer|""movie""|""filmselect""|""The LEGO Movie 2...",3128325,38937,1198,6580,https://i.ytimg.com/vi/11K013qpRR4/default.jpg,False,False,False,The LEGO Movie 2 Trailer


## Buscando o nome da categoria pelo identificador e armazenando em nova coluna

In [5]:
with open(category_filename) as f:
    category = json.load(f)

In [6]:
def category_replace(c_id):
    for i in category["items"]:
        if int(c_id) == int(i["id"]):
             return i["snippet"]["title"]
    return "None"

In [7]:
videos_lastentry['category_name'] = videos_lastentry.apply(lambda row: category_replace(row['category_id']), axis=1)
print(filename+" shape: "+str(videos_lastentry.shape))
videos_lastentry.head()

GBvideos.csv shape: (3217, 18)


,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name
38915,4YFo4bdMO8Q,2018-06-14,KYLE - Ikuyo feat. 2 Chainz & Sophia Black [A...,SuperDuperKyle,2018-05-11,10,04:06:35,"Kyle|""SuperDuperKyle""|""Ikuyo""|""2 Chainz""|""Soph...",607552,18271,274,1423,https://i.ytimg.com/vi/4YFo4bdMO8Q/default.jpg,False,False,False,Debut album 'Light of Mine' out now: http://ky...,Music
38811,3KmIMuLh4Vs,2018-06-14,Interpol - The Rover,InterpolVEVO,2018-06-07,10,16:29:06,"Interpol|""The Rover""|""Marauder""|""Matador""|""Mat...",442492,17004,178,1670,https://i.ytimg.com/vi/3KmIMuLh4Vs/default.jpg,False,False,False,'The Rover' is the first single taken from Int...,Music
38809,CWSYOdxx8Tg,2018-06-14,How To Make a Hit Football Anthem,Brett Domino,2018-06-07,23,07:30:00,"brett domino|""Umbro""|""England Anthem""|""World C...",56138,2814,73,170,https://i.ytimg.com/vi/CWSYOdxx8Tg/default.jpg,False,False,False,Brett Domino and Umbro’s guide to writing a hi...,Comedy
38808,RtvUoab88UE,2018-06-14,Dear Ms. Banks,Bob The Drag Queen,2018-06-06,22,22:27:21,"bob the drag queen|""drag""|""diss track""|""rap""|""...",169099,13780,795,2084,https://i.ytimg.com/vi/RtvUoab88UE/default.jpg,False,False,False,Thanks to Mitch Ferrino for helping with audio.,People & Blogs
38807,11K013qpRR4,2018-06-14,The LEGO Movie 2 Trailer (2019),FilmSelect Trailer,2018-06-05,24,12:50:30,"trailer|""movie""|""filmselect""|""The LEGO Movie 2...",3128325,38937,1198,6580,https://i.ytimg.com/vi/11K013qpRR4/default.jpg,False,False,False,The LEGO Movie 2 Trailer,Entertainment


## Transformando categorias em OneHotEncoder

In [8]:
# DESNECESSARIO - UTILIZADO ONEHOTENCODER() NO MAPPER
#videos_lastentry = videos_lastentry.join(pd.get_dummies(videos_lastentry['category_name']))
#videos_lastentry.head()

## Definindo sentimento por video

In [9]:
#positive sentiment = 60% da quantidade de likes+dislikes
videos_lastentry['sentiment'] = videos_lastentry['likes'] >= 0.6*(videos_lastentry['likes']+videos_lastentry['dislikes']) 
videos_lastentry.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,sentiment
38915,4YFo4bdMO8Q,2018-06-14,KYLE - Ikuyo feat. 2 Chainz & Sophia Black [A...,SuperDuperKyle,2018-05-11,10,04:06:35,"Kyle|""SuperDuperKyle""|""Ikuyo""|""2 Chainz""|""Soph...",607552,18271,274,1423,https://i.ytimg.com/vi/4YFo4bdMO8Q/default.jpg,False,False,False,Debut album 'Light of Mine' out now: http://ky...,Music,True
38811,3KmIMuLh4Vs,2018-06-14,Interpol - The Rover,InterpolVEVO,2018-06-07,10,16:29:06,"Interpol|""The Rover""|""Marauder""|""Matador""|""Mat...",442492,17004,178,1670,https://i.ytimg.com/vi/3KmIMuLh4Vs/default.jpg,False,False,False,'The Rover' is the first single taken from Int...,Music,True
38809,CWSYOdxx8Tg,2018-06-14,How To Make a Hit Football Anthem,Brett Domino,2018-06-07,23,07:30:00,"brett domino|""Umbro""|""England Anthem""|""World C...",56138,2814,73,170,https://i.ytimg.com/vi/CWSYOdxx8Tg/default.jpg,False,False,False,Brett Domino and Umbro’s guide to writing a hi...,Comedy,True
38808,RtvUoab88UE,2018-06-14,Dear Ms. Banks,Bob The Drag Queen,2018-06-06,22,22:27:21,"bob the drag queen|""drag""|""diss track""|""rap""|""...",169099,13780,795,2084,https://i.ytimg.com/vi/RtvUoab88UE/default.jpg,False,False,False,Thanks to Mitch Ferrino for helping with audio.,People & Blogs,True
38807,11K013qpRR4,2018-06-14,The LEGO Movie 2 Trailer (2019),FilmSelect Trailer,2018-06-05,24,12:50:30,"trailer|""movie""|""filmselect""|""The LEGO Movie 2...",3128325,38937,1198,6580,https://i.ytimg.com/vi/11K013qpRR4/default.jpg,False,False,False,The LEGO Movie 2 Trailer,Entertainment,True


## Normalização das features numéricas

In [10]:
videos_lastentry.describe()

,category_id,views,likes,dislikes,comment_count
count,3217.000000,3.217000e+03,3.217000e+03,3.217000e+03,3.217000e+03
mean,17.944047,4.858976e+06,1.005430e+05,5.355717e+03,9.631431e+03
std,7.540560,1.939526e+07,3.169082e+05,3.747721e+04,4.129257e+04
min,1.000000,1.014000e+03,0.000000e+00,0.000000e+00,0.000000e+00
25%,10.000000,1.661840e+05,3.764000e+03,1.350000e+02,4.390000e+02
50%,22.000000,6.120910e+05,1.369400e+04,5.040000e+02,1.522000e+03
75%,24.000000,2.455913e+06,6.083000e+04,2.208000e+03,5.540000e+03
max,43.000000,4.245389e+08,5.613827e+06,1.753274e+06,1.228655e+06


In [11]:
# Normalização
videos_lastentry['views_nonnormal'] = videos_lastentry['views']
videos_lastentry['views'] = (videos_lastentry['views']-videos_lastentry['views'].min())/(videos_lastentry['views'].max()-videos_lastentry['views'].min())
#videos_lastentry['likes'] = (videos_lastentry['likes']-videos_lastentry['likes'].min())/(videos_lastentry['likes'].max()-videos_lastentry['likes'].min())
#videos_lastentry['dislikes'] = (videos_lastentry['dislikes']-videos_lastentry['dislikes'].min())/(videos_lastentry['dislikes'].max()-videos_lastentry['dislikes'].min())
videos_lastentry['comment_count'] = (videos_lastentry['comment_count']-videos_lastentry['comment_count'].min())/(videos_lastentry['comment_count'].max()-videos_lastentry['comment_count'].min())
videos_lastentry.describe()

,category_id,views,likes,dislikes,comment_count,views_nonnormal
count,3217.000000,3217.000000,3.217000e+03,3.217000e+03,3217.000000,3.217000e+03
mean,17.944047,0.011443,1.005430e+05,5.355717e+03,0.007839,4.858976e+06
std,7.540560,0.045686,3.169082e+05,3.747721e+04,0.033608,1.939526e+07
min,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,1.014000e+03
25%,10.000000,0.000389,3.764000e+03,1.350000e+02,0.000357,1.661840e+05
50%,22.000000,0.001439,1.369400e+04,5.040000e+02,0.001239,6.120910e+05
75%,24.000000,0.005783,6.083000e+04,2.208000e+03,0.004509,2.455913e+06
max,43.000000,1.000000,5.613827e+06,1.753274e+06,1.000000,4.245389e+08


## Separando conjuntos de treino, validação e teste

In [12]:
train_data = videos_lastentry.sample(frac=0.8, random_state=200)
test_data = videos_lastentry.drop(train_data.index)
print(train_data.shape, test_data.shape)

(2574, 20) (643, 20)


# Classificador

## União de features

In [13]:
mapper = DataFrameMapper([
    ('title', CountVectorizer()),
    ('description', CountVectorizer(stop_words='english', max_features = 5000, strip_accents='ascii')),
    ('tags', CountVectorizer()),
    ('category_name', LabelBinarizer()),
    ('comments_disabled', None),
    ('views', None),
    ('comment_count', None)
])
x_train = mapper.fit_transform(train_data)
y_train = train_data['sentiment'].values

In [14]:
x_train.shape

(2574, 24870)

In [15]:
classifier = LogisticRegression(penalty = 'l1')
classifier.fit(x_train, y_train)

c:\users\work\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [16]:
x_test = mapper.transform(test_data)
y_test = test_data['sentiment'].values
predicted = classifier.predict(x_test)

In [17]:
probs = classifier.predict_proba(x_test)
print(probs)

[[6.32813826e-08 9.99999937e-01]
 [4.96533515e-04 9.99503466e-01]
 [2.54849507e-03 9.97451505e-01]
 ...
 [5.26287953e-03 9.94737120e-01]
 [5.39427625e-04 9.99460572e-01]
 [4.19460256e-05 9.99958054e-01]]


In [18]:
print("Acuracia: "+str(metrics.accuracy_score(y_test, predicted)))
print("ROC AUC score: "+str(metrics.roc_auc_score(y_test, probs[:, 1])))

Acuracia: 0.9797822706065319
ROC AUC score: 0.7585182250396196


In [19]:
predicted = classifier.predict(mapper.transform(videos_lastentry))

In [20]:
videos_output = videos_lastentry
videos_output['predicted_sentiment'] = predicted

videos_output = videos_output.sort_values('predicted_sentiment', ascending=False)
videos_output.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,...,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,sentiment,views_nonnormal,predicted_sentiment
38915,4YFo4bdMO8Q,2018-06-14,KYLE - Ikuyo feat. 2 Chainz & Sophia Black [A...,SuperDuperKyle,2018-05-11,10,04:06:35,"Kyle|""SuperDuperKyle""|""Ikuyo""|""2 Chainz""|""Soph...",0.001429,18271,...,0.001158,https://i.ytimg.com/vi/4YFo4bdMO8Q/default.jpg,False,False,False,Debut album 'Light of Mine' out now: http://ky...,Music,True,607552,True
11094,s9EE_r0WRMk,2018-01-08,What I Got For CHRISTMAS! | Fleur De Force,Fleur DeForce,2017-12-27,26,17:00:11,"fleurdeforce|""fleur de force""|""fleurdevlog""|""f...",0.000742,11317,...,0.000413,https://i.ytimg.com/vi/s9EE_r0WRMk/default.jpg,False,False,False,What I got for CHRISTMAS this year! I had the ...,Howto & Style,True,316143,True
11337,1Wo6WEBhBBw,2018-01-09,"Syrian War Report – December 25, 2017: Army Cu...",South Front,2017-12-25,25,14:08:40,"Syrian War Report|""Syrian War Update""|""Syrian ...",0.000087,1007,...,0.000187,https://i.ytimg.com/vi/1Wo6WEBhBBw/default.jpg,False,False,False,"If you’re able, and if you like our content an...",News & Politics,True,37743,True
11336,sXP6vliZIHI,2018-01-09,Cardi B - Bartier Cardi (feat. 21 Savage) [Off...,Cardi B,2017-12-22,22,05:00:02,"Cardi B|""Bardi Gang""|""Bardi B""|""Bartier Cardi""...",0.058064,443444,...,0.026085,https://i.ytimg.com/vi/sXP6vliZIHI/default.jpg,False,False,False,Cardi B - Bartier Cardi (feat. 21 Savage) avai...,People & Blogs,True,24651559,True
11342,ItL6vcUrpUs,2018-01-09,MACKLEMORE FEAT DAVE B & TRAVIS THOMPSON - COR...,Macklemore LLC,2017-12-21,10,17:00:20,"Mackelmore|""Thrift Shop""|""Cant Hold Us""|""Can't...",0.014519,160541,...,0.004047,https://i.ytimg.com/vi/ItL6vcUrpUs/default.jpg,False,False,False,GEMINI - AVAILABLE NOW\nhttp://smarturl.it/Mac...,Music,True,6164967,True


In [21]:
f1_score(videos_output['sentiment'].astype(int), videos_output['predicted_sentiment'].astype(int), average='macro')

0.8272931514961224

In [22]:
get_n_features = 10
print("The "+str(get_n_features)+" most positive-weighted words are: ")
for feature_id in sorted(range(len(classifier.coef_[0])), key=lambda i: classifier.coef_[0][i], reverse=True)[:get_n_features]:
    print("%s (%f)" % (mapper.transformed_names_[feature_id], classifier.coef_[0][feature_id]))
print()
print("The "+str(get_n_features)+" most negative-weighted words are: ")
for feature_id in sorted(range(len(classifier.coef_[0])), key=lambda i: classifier.coef_[0][i], reverse=False)[:get_n_features]:
    print("%s (%f)" % (mapper.transformed_names_[feature_id], classifier.coef_[0][feature_id]))

The 10 most positive-weighted words are: 
description_nhttp (0.698018)
title_the (0.677737)
title_in (0.556447)
description_nhttps (0.545247)
description_videos (0.508302)
tags_rapper (0.466220)
tags_funny (0.428830)
description_star (0.428083)
description_movie (0.416276)
description_music (0.397416)

The 10 most negative-weighted words are: 
title_backlash (-4.025130)
tags_caller (-1.823142)
tags_variety (-1.697936)
tags_schumer (-1.658201)
description_comedy (-1.566773)
tags_natural (-1.517953)
description_certain (-1.516578)
title_campaign (-1.496177)
description_katie (-1.303159)
title_conor (-1.288168)


# K-Fold

In [23]:
def run_kfold(clf,X,Y):
    kf = KFold(n_splits=10)
    outcomes = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        outcomes.append(accuracy)
    mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome)) 

In [24]:
run_kfold(classifier,x_train,y_train)

c:\users\work\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\work\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\work\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\work\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\work\appdata\local\programs\python\python37\lib\site-pa

Mean Accuracy: 0.9766898923174374


In [25]:
videos_lastentry['like_ratio'] = videos_lastentry['likes']/(videos_lastentry['likes']+videos_lastentry['dislikes'])

In [26]:
from IPython.display import HTML, display

# We choose the 10 most trending videos
selected_columns = ['title', 'channel_title', 'thumbnail_link', 'publish_date', 'category_name', 'likes', 'dislikes', 'views_nonnormal', 'like_ratio']

most_frequent = videos_lastentry.groupby(selected_columns)['video_id'].agg(
    {"code_count": len}).sort_values(by=['like_ratio']
).head(10).reset_index()

# Construction of HTML table with miniature photos assigned to the most popular movies
table_content = ''
max_title_length = 50

for date, row in most_frequent.T.iteritems():
    HTML_row = '<tr>'
    HTML_row += '<td><img src="' + str(row[2]) + '"style="width:100px;height:100px;"></td>'
    HTML_row += '<td>' + str(row[1]) + '</td>'
    HTML_row += '<td>' + str(row[0])  + '</td>'
    HTML_row += '<td>' + str(row[4]) + '</td>'
    HTML_row += '<td>' + str(row[3]) + '</td>'
    HTML_row += '<td>' + str(row[5]) + '</td>'
    HTML_row += '<td>' + str(row[6]) + '</td>'
    HTML_row += '<td>' + str(row[7]) + '</td>'
    
    table_content += HTML_row + '</tr>'

display(HTML(
    '<table><tr><th>Photo</th><th>Channel Name</th><th style="width:250px;">Title</th><th>Category</th><th>Publish Date</th><th>Likes</th><th>Dislikes</th><th>Views</th></tr>{}</table>'.format(table_content))
)

c:\users\work\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  import sys


Photo,Channel Name,Title,Category,Publish Date,Likes,Dislikes,Views
,RollerCoaster Tycoon,RollerCoaster Tycoon Start Engine Campaign,Gaming,2018-01-23 00:00:00,136,3780,54100
,Daily Caller,PSA from Chairman of the FCC Ajit Pai,People & Blogs,2017-12-13 00:00:00,10576,258504,1348067
,Roy Moore for Senate,Judge Roy Moore Campaign Statement,People & Blogs,2017-12-14 00:00:00,4372,59691,804935
,ReBoot: The Guardian Code,ReBoot: The Guardian Code Official Trailer,Film & Animation,2018-02-22 00:00:00,970,11793,326871
,F&F Clothing,F&F Active | #whatsstoppingyou,Howto & Style,2018-01-01 00:00:00,5,49,3571
,NBC News,Roy Moore Delivers Concession Speech From Alabama (Full) | NBC News,News & Politics,2017-12-13 00:00:00,631,4807,209811
,Saturday Night Live,Amy Schumer Is Ready to Host the Best SNL Yet,Entertainment,2018-05-11 00:00:00,1084,5790,112317
,Rapid Trailer,HEATHERS Official Trailer (2018) Comedy TV Show HD,Film & Animation,2018-01-18 00:00:00,2731,14150,265512
,Ryan Burger,Billy Mitchell Statement,People & Blogs,2018-04-15 00:00:00,521,2465,240891
,paulitaale,Volar - Jaime Ciero,Music,2011-12-26 00:00:00,1416,5939,1674208


In [27]:
videos_lastentry.loc[videos_lastentry['title'].str.contains("backlash")]

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,...,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,sentiment,views_nonnormal,predicted_sentiment,like_ratio
9163,lYEj9snU838,2017-12-29,"Boy speaks out on viral bullying video, mom ad...",CBS This Morning,2017-12-12,25,12:58:41,"video|""cbs""|""news""|""Keaton Jones""|""Tennessee""|...",0.003665,10164,...,https://i.ytimg.com/vi/lYEj9snU838/default.jpg,False,False,False,"Keaton Jones, a Tennessee middle schooler, is ...",News & Politics,False,1556796,False,0.350579
